In [1]:
!pip install google-genai

In [2]:
from google.colab import userdata
import os
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [3]:
import textwrap
from google import genai
import os

try:
    client = genai.Client()
except Exception as e:
    print(f"Error initializing client. Make sure GEMINI_API_KEY is set correctly. Error: {e}")
    client = None

def get_llm_response_real(prompt, technique_name):
    """
    Sends the prompt to the Gemini model and returns the response text.
    """
    if client is None:
        return f"[[API Client Not Initialized. Check your API Key.]]"

    print(f"\n--- {technique_name.upper()} PROMPT ---")
    print(textwrap.fill(prompt, width=80))
    print("--------------------------------------------------------------------------------")

    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt
        )
        return response.text
    except Exception as e:
        return f"[[API Call Failed for {technique_name}: {e}]]"

TASK_QUESTION = (
    "A baker made 30 cupcakes. He sold 15 in the morning. "
    "In the afternoon, he made 2 more batches of 10 cupcakes each. "
    "How many cupcakes does the baker have now?"
)

# --- 1. ZERO-SHOT PROMPT ---
zero_shot_prompt = (
    "Answer the following question. "
    "You must only provide the final numerical answer. "
    f"Question: {TASK_QUESTION}"
)
zero_shot_output = get_llm_response_real(zero_shot_prompt, "Zero-Shot")
print(f"**ZERO-SHOT OUTPUT:**\n{zero_shot_output}")
print("\n" + "="*80)


# --- 2. FEW-SHOT PROMPT ---
few_shot_prompt = textwrap.dedent(f"""
    Determine the total amount based on the provided scenario. Provide the calculation followed by the answer.

    Example 1:
    Question: A gardener planted 5 roses. She then planted 3 more. Later, a deer ate 2. How many roses are left?
    Answer: 5 + 3 - 2 = 6

    Example 2:
    Question: A store had 10 shirts. They sold 4. They then received a new shipment of 20 shirts. What is the total number of shirts now?
    Answer: 10 - 4 + 20 = 26

    Question: {TASK_QUESTION}
    Answer:
""")
few_shot_output = get_llm_response_real(few_shot_prompt, "Few-Shot")
print(f"**FEW-SHOT OUTPUT:**\n{few_shot_output}")
print("\n" + "="*80)


cot_prompt = (
    "Answer the following question. "
    "First, reason through the problem step-by-step to explain your logic. "
    "Then, provide the final answer on a new line labeled 'Final Answer:'. "
    f"Question: {TASK_QUESTION}\n"
    "Let's think step by step."
)
cot_output = get_llm_response_real(cot_prompt, "Chain-of-Thought")
print(f"**CHAIN-OF-THOUGHT OUTPUT:**\n{cot_output}")
print("\n" + "="*80)


--- ZERO-SHOT PROMPT ---
Answer the following question. You must only provide the final numerical answer.
Question: A baker made 30 cupcakes. He sold 15 in the morning. In the afternoon,
he made 2 more batches of 10 cupcakes each. How many cupcakes does the baker
have now?
--------------------------------------------------------------------------------
**ZERO-SHOT OUTPUT:**
35


--- FEW-SHOT PROMPT ---
 Determine the total amount based on the provided scenario. Provide the
calculation followed by the answer.  Example 1: Question: A gardener planted 5
roses. She then planted 3 more. Later, a deer ate 2. How many roses are left?
Answer: 5 + 3 - 2 = 6  Example 2: Question: A store had 10 shirts. They sold 4.
They then received a new shipment of 20 shirts. What is the total number of
shirts now? Answer: 10 - 4 + 20 = 26  Question: A baker made 30 cupcakes. He
sold 15 in the morning. In the afternoon, he made 2 more batches of 10 cupcakes
each. How many cupcakes does the baker have now? An